# C
## First
$$r_0 = \int_{0}^{1} \frac{1}{1+x}dx=ln(X+1)|^1_0=ln2\\
\because \int^1_0 \sum^j_{i=1}(-1)^{i-1}x^{i-1}dx = \int^1_0 \frac{1}{1+x}dx-(-1)^jr_j\\
\therefore \sum^j_{i=1}\frac{1}{i} = ln2-(-1)^jr_j\\
r_j = (-1)^jln2+\sum^j_{i=1}(-1)^{i+j}\frac{1}{i}
$$

we have
$ H\vec{\alpha} = \vec{r}=\vec{\beta}*ln2+\vec{\gamma}$

it is known that $H_n^{-1}=[b_{ij}]$
$$
[bij]=[\frac{(−1)^{i+j}(i+n−1)!(j+n−1)!}{((i−1)!)^2((j−1)!)^2(n−j)!(n−i)!(i+j−1)}]
$$
each element is rational number, thus
$$
\vec{\alpha} = H^{-1}*\vec{\beta}*ln2+H^{-1}*\vec{\gamma}
$$
has the required form.

## Second


In [66]:
import numpy as np
from fractions import Fraction

# construct a Hilbert matrix
def Hilbert(n,fraction=0):#创建Hilbert矩阵
    if fraction==1:
        A=np.zeros((n,n),dtype=Fraction)
        for i in range(n):
            for j in range(n):
                A[i][j]=Fraction(1,(i+j+1))
    else:
        A=np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                A[i][j]=1/(i+j+1)
    return A

# Gauss elimination
def Gauss(A_in,b,fraction=0):
    n = b.shape[0]
    A = A_in.copy()
    #消元
    for k in range(n-1):    #k第一层循环，第（0~n-1）行
        for i in range(k+1,n):    #i第二层循环，第（k+1,n）行
            mik = A[i,k]/A[k,k]
            for j in range(k+1,n):    
                A[i,j] = A[i,j] - mik*A[k,j]
            b[i] = b[i] - mik*b[k]
    #回代
    if fraction==1:
        solution = np.zeros(n,dtype=Fraction)
    else:
        solution = np.zeros(n)
    solution[n-1] = b[n-1]/A[n-1,n-1]    #改为n-1
    for i in range(n-2,-1,-1):
        for j in range(i+1,n):
            solution[i] = solution[i] + A[i,j]*solution[j]
        solution[i] = (b[i] - solution[i])/A[i,i]
    return solution

# beta = H^-1*{(-1)^j}_n, n is the size of H
def beta(n):
    b = np.zeros(n,dtype=Fraction)
    for i in range(n):
        b[i] = Fraction((-1)**i,1)
    return b

# gamma = H^-1*{\sum_{i=1}^j (-1)^{i+j}1/i}_n
def gamma(n):
    g = np.zeros(n,dtype=Fraction)
    for i in range(1,n):
        for j in range(1,i+1):
            g[i] += Fraction((-1)**(i+j),j)
    return g


## Third

In [67]:
for n in range(1,7):
    print("n = ",n)
    # nth order polynomial needs n+1 order Hilbert matrix
    H = Hilbert(n+1,1)
    #print("H = ",H)
    b = beta(n+1)
    #print("b = ",b)
    g = gamma(n+1)
    #print("g = ",g)
    x1 = Gauss(H,b,1).astype(float)
    #print("x1 = ",x1)
    x2 = Gauss(H,g,1).astype(float)
    #print("x2 = ",x2)
    alpha = x1*np.log(2)+x2
    
    print("alpha = ",alpha)

n =  1
alpha =  [ 0.93147181 -0.47664925]
n =  2
alpha =  [ 0.98603854 -0.80404967  0.32740042]
n =  3
alpha =  [ 0.9972785  -0.93892919  0.66459923 -0.22479921]
n =  4
alpha =  [ 0.99948314 -0.98302195  0.86301662 -0.53344848  0.15432464]
n =  5
alpha =  [ 0.99990351 -0.99563313  0.95129493 -0.76885732  0.41915959 -0.10593398]
n =  6
alpha =  [ 0.99998221 -0.99893826  0.98434622 -0.90106246  0.66704419 -0.32407242
  0.07271281]


## Fourth

In [69]:
def get_r(n):
    r = np.zeros(n)
    for i in range(n):
        r[i] += (-1)**i*np.log(2)
        for j in range(1,i+1):
            r[i] += (-1)**(i+j)/j
    return r

for n in range(1,7):
    # nth order polynomial needs n+1 order Hilbert matrix
    H = Hilbert(n+1)
    r = get_r(n+1)
    alpha = np.dot(np.linalg.inv(H),r)#alpha = np.linalg.solve(H,r)
    ki = np.linalg.cond(H)
    print("n = ",n)
    print("ki = ",ki)
    print("alpha = ",alpha)

n =  1
ki =  19.281470067903967
alpha =  [ 0.93147181 -0.47664925]
n =  2
ki =  524.0567775860627
alpha =  [ 0.98603854 -0.80404967  0.32740042]
n =  3
ki =  15513.738738929103
alpha =  [ 0.9972785  -0.93892919  0.66459923 -0.22479921]
n =  4
ki =  476607.25024100044
alpha =  [ 0.99948314 -0.98302195  0.86301662 -0.53344848  0.15432464]
n =  5
ki =  14951058.641453395
alpha =  [ 0.99990351 -0.99563313  0.95129493 -0.76885732  0.41915959 -0.10593398]
n =  6
ki =  475367356.63047224
alpha =  [ 0.99998221 -0.99893826  0.98434621 -0.90106245  0.66704419 -0.32407243
  0.07271282]


as seen from above, in Fourth directly calculation of $H^{-1}r$ instead of precise calculation of fractions is supposed to be bad, because H has crazy condition numbers.

maybe Numpy has optimized Hilbert matrix in np.linalg.inv, the accuracy in Fourth is almost the same.

## Finally